Setup

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.decomposition import PCA
from tensorflow.keras import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import TensorBoard

# Create a TensorBoard callback
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1)

Read and transform the data

In [8]:
# load dataset
CCData = pd.read_csv("Credit_card.csv")
CCData2 = pd.read_csv("Credit_card_label.csv")
X = CCData[["Ind_ID","GENDER","Car_Owner","Propert_Owner","CHILDREN","Annual_income","Type_Income","EDUCATION",
        "Marital_status","Housing_type","Birthday_count","Employed_days","Mobile_phone","Work_Phone","Phone",
        "EMAIL_ID","Type_Occupation","Family_Members"]]
Y = CCData2[["label"]]

#Find and replace all 'NaN data
#Columns with string data get "N/A"
#Columns with numbered data get 0
listNan = X.loc[:, X.isna().any()]
for title in list(listNan):
  if X[title].dtype == 'object':
    X[title] = X[title].fillna('N/A')
  else:
    X[title] = X[title].fillna(0)

#Encode the catagorical data
le = LabelEncoder()
string_columns = X.select_dtypes(include=['object']).columns
for title in list(string_columns):
  X[title] = le.fit_transform(X[title].values)

# Standardizing the features and perform pca
X = StandardScaler().fit_transform(X)
pca = PCA(n_components=5)
principalComponents = pca.fit_transform(X)

# Split the dataset into training and test sets
X, X_test, Y, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler().fit(X)
X = scaler.transform(X)
X_test = scaler.transform(X_test)

#Turn Y into a matrix
Y = np.ravel(Y)
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)

Create and train the model

In [9]:
avg_accuracy = 0
for i in range(10):
  #Create the model
  model = Sequential()
  model.add(Dense(16, input_dim=X.shape[1], activation='sigmoid'))
  model.add(Dense(16, activation='sigmoid'))
  model.add(Dense(4, activation='sigmoid'))
  #model.add(Dense(2, activation='sigmoid'))  # Output layer for 'sparse_categorical_crossentropy'
  model.add(Dense(1, activation='sigmoid'))  # Output layer for all of the others
  model.compile(loss='mse',
                optimizer=Adam(learning_rate=0.05),
                metrics=['Accuracy'])
  #Training the model
  history = model.fit(X, Y, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[tensorboard])
  avg_accuracy = avg_accuracy + model.evaluate(X_test,y_test)[1]
avg_accuracy = avg_accuracy/10
print(avg_accuracy)

Epoch 1/10
39/39 [==============================] - 2s 15ms/step - loss: 0.1533 - Accuracy: 0.8231 - val_loss: 0.0951 - val_Accuracy: 0.9032
Epoch 2/10
39/39 [==============================] - 0s 9ms/step - loss: 0.1039 - Accuracy: 0.8829 - val_loss: 0.0874 - val_Accuracy: 0.9032
Epoch 3/10
39/39 [==============================] - 0s 9ms/step - loss: 0.1035 - Accuracy: 0.8829 - val_loss: 0.0884 - val_Accuracy: 0.9032
Epoch 4/10
39/39 [==============================] - 0s 10ms/step - loss: 0.1038 - Accuracy: 0.8829 - val_loss: 0.0871 - val_Accuracy: 0.9032
Epoch 5/10
39/39 [==============================] - 0s 5ms/step - loss: 0.1031 - Accuracy: 0.8829 - val_loss: 0.0870 - val_Accuracy: 0.9032
Epoch 6/10
39/39 [==============================] - 0s 5ms/step - loss: 0.1027 - Accuracy: 0.8829 - val_loss: 0.0869 - val_Accuracy: 0.9032
Epoch 7/10
39/39 [==============================] - 0s 5ms/step - loss: 0.1021 - Accuracy: 0.8829 - val_loss: 0.0865 - val_Accuracy: 0.9032
Epoch 8/10
39/39 [

**1: 2 componants with learning rate = 0.01**
1.   loss='sparse_categorical_crossentropy', Accuracy = 0.903
2.   loss='mse', Accuracy = 0.903
3.   loss='msle', Accuracy = 0.903
4.   loss = 'poisson', Accuracy = 0.903
5.   loss='binary_crossentropy', Accuracy = 0.903

**2: 3 componants with learning rate = 0.01**
1.   loss='mse', Accuracy = 0.8829




